#COS 575 Lab 5

##Tatiana Romanchishina

Using the chr2015 data set, fit classification models in order to predict whether a given community has a Premature_death_value above or below the median. 
Explore logistic regression, LDA, and KNN (select a few models with different values for k) models using at least three subsets of predictors with up to a max of five predictors. Be sure to produce a confusion matrix for each of the techniques. Describe your findings. Which subset produced the best results ?

In [1]:
# load numpy and pandas for data manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#%pylab inline

from pylab import *

# load statsmodels
import statsmodels.api as sm

from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.lda import LDA
from sklearn.neighbors import KNeighborsClassifier

from collections import OrderedDict
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix as sk_confusion_matrix
from patsy import dmatrices
from sklearn.cross_validation import cross_val_score
from sklearn import preprocessing


# load the data
path = r'CHR2015.csv'
df = pd.read_csv(path)
df.head()

,STATECODE,COUNTYCODE,State,County,Premature_death_Value,Poor_or_fair_health_Value,Poor_physical_health_days_Value,Poor_mental_health_days_Value,Low_birthweight_Value,Adult_smoking_Value,...,Uninsured_adults_Value,Uninsured_children_Value,Health_care_costs_Value,Could_not_see_doctor_due_to_cost_Value,Other_primary_care_providers_Value,Median_household_income_Value,Children_eligible_for_free_lunch_Value,Homicide_rate_Value,bottom10,top10
0,1,0,AL,Alabama,9508,0.205,4.3,4.3,0.10,0.219,...,0.202,0.043,10127,0.163,48,42882,0.500,9.1,0,0
1,1,1,AL,Autauga County,8405,0.228,5.1,3.6,0.09,0.217,...,0.169,0.037,9939,0.156,18,51868,0.383,4.6,0,0
2,1,3,AL,Baldwin County,7457,0.127,3.3,3.8,0.09,0.206,...,0.199,0.054,9502,0.144,29,47539,0.344,4.3,0,0
3,1,5,AL,Barbour County,8901,0.234,4.8,4.3,0.12,0.251,...,0.228,0.045,10414,0.169,11,30981,0.697,NaN,0,0
4,1,7,AL,Bibb County,10729,0.179,4.7,5.1,0.13,0.259,...,0.193,0.042,10825,0.163,9,39781,0.546,7.8,0,0


In [2]:
# add the response column 
df['aboveMedianPDV'] = [1 if d <= np.median(df['Premature_death_Value']) else 0 for d in df['Premature_death_Value']]
print(len(df[df.aboveMedianPDV == 0]), len(df[df.aboveMedianPDV == 1]), df.shape)

1524 1524 (3048, 71)


In [42]:
# count the null values in all columns to decide on whether it would be 
# beneficial to remove some of them to preserve most rows
#df.isnull().sum()

In [8]:
# remove NaNs
df_fix = df[~np.isnan(df.Adult_smoking_Value) & ~np.isnan(df.Adult_obesity_Value) &
           ~np.isnan(df.Physical_inactivity_Value) & ~np.isnan(df.Diabetes_Value) &
            ~np.isnan(df.Teen_births_Value) & ~np.isnan(df.aboveMedianPDV)]
df_fix = df_fix.reset_index(drop=True)
df_fix[['Adult_smoking_Value', 'Adult_obesity_Value', 'Physical_inactivity_Value', 
             'Diabetes_Value', 'Teen_births_Value']].isnull().sum()

Adult_smoking_Value          0
Adult_obesity_Value          0
Physical_inactivity_Value    0
Diabetes_Value               0
Teen_births_Value            0
dtype: int64

In [9]:
# create dataframes with an intercept column
y, X = dmatrices('aboveMedianPDV ~ Adult_smoking_Value + Adult_obesity_Value + \
                 Physical_inactivity_Value + Diabetes_Value + \
                 Teen_births_Value',
                df_fix, return_type="dataframe")
# flatten y into a 1-D array
#y = np.ravel(y) #OR
y = y.values.flatten()

###Logistic Regression

In [53]:
# instantiate a logistic regression model, and fit with X and y
model = LogisticRegression()
model = model.fit(X, y)

# check the score
model.score(X, y)

0.8412874583795783

In [24]:
# what percentage had above median PDV
y.mean()

0.50055493895671477

In [31]:
# evaluate the model by splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=1)
model2 = LogisticRegression()
model2.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [32]:
# predict class labels for the test set
predicted = model2.predict(X_test)
print (predicted)

[ 1.  1.  1. ...,  1.  0.  0.]


In [33]:
# generate class probabilities
probs = model2.predict_proba(X_test)
print (probs)

[[ 0.02240709  0.97759291]
 [ 0.0854317   0.9145683 ]
 [ 0.04864101  0.95135899]
 ..., 
 [ 0.48537846  0.51462154]
 [ 0.63218017  0.36781983]
 [ 0.83999376  0.16000624]]


In [34]:
# generate evaluation metrics
print (metrics.accuracy_score(y_test, predicted))
#print (metrics.roc_auc_score(y_test, probs[:, 1]))

0.824399260628


In [36]:
# print confusion matrix
print (metrics.confusion_matrix(y_test, predicted))
print (metrics.classification_report(y_test, predicted))

[[432 102]
 [ 88 460]]
             precision    recall  f1-score   support

        0.0       0.83      0.81      0.82       534
        1.0       0.82      0.84      0.83       548

avg / total       0.82      0.82      0.82      1082



In [40]:
# evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10)
print (scores)
print (scores.mean())

[ 0.87084871  0.7896679   0.74169742  0.86666667  0.85555556  0.81481481
  0.81111111  0.84444444  0.85555556  0.81481481]
0.82651769851


###LDA

In [33]:
preprocessing.scale(X, axis=0, with_mean=True, with_std=True, copy=False)

array([[  0.00000000e+00,   9.25724549e-02,   5.36061051e-01,
          4.52388003e-01,   8.03939257e-01,   2.26519799e-01],
       [  0.00000000e+00,   6.05725386e-02,   1.14802847e-01,
          1.81452368e-01,   4.67677772e-01,  -3.15596377e-01],
       [  0.00000000e+00,  -1.15427001e-01,  -1.28200067e+00,
         -3.06231776e-01,   2.15481659e-01,   1.09592781e-01],
       ..., 
       [  0.00000000e+00,  -5.95425746e-01,  -3.28626841e-01,
         -4.50730782e-01,  -1.17159697e+00,   3.29549125e-03],
       [  0.00000000e+00,  -1.17142424e+00,  -1.45937255e+00,
         -5.22980284e-01,  -5.41106682e-01,  -1.45520714e-01],
       [  0.00000000e+00,  -2.43426666e-01,  -4.39484263e-01,
          8.28610650e-04,  -1.04549891e+00,   1.25537374e-01]])

In [34]:
# supress all warnings because LDA warns about collinearity 
import warnings
warnings.filterwarnings("ignore")

In [62]:
# LDA
model = LDA()
model = model.fit(X, y)

# check the score
model.score(X, y)

0.84387717351091385

In [55]:
# evaluate the model by splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=1)
model2 = LDA()
model2.fit(X_train, y_train)

LDA(n_components=None, priors=None, shrinkage=None, solver='svd',
  store_covariance=False, tol=0.0001)

In [56]:
# predict class labels for the test set
predicted = model2.predict(X_test)
print (predicted)

[ 1.  1.  1. ...,  0.  0.  0.]


In [57]:
# generate class probabilities
probs = model2.predict_proba(X_test)
print (probs)

[[ 0.01069505  0.98930495]
 [ 0.0438153   0.9561847 ]
 [ 0.01643779  0.98356221]
 ..., 
 [ 0.57334576  0.42665424]
 [ 0.90817245  0.09182755]
 [ 0.93983026  0.06016974]]


In [58]:
# generate evaluation metrics
print (metrics.accuracy_score(y_test, predicted))
print (metrics.roc_auc_score(y_test, probs[:, 1]))

0.846580406654
0.92162511277


In [59]:
# print confusion matrix
print (metrics.confusion_matrix(y_test, predicted))
print (metrics.classification_report(y_test, predicted))

[[442  92]
 [ 74 474]]
             precision    recall  f1-score   support

        0.0       0.86      0.83      0.84       534
        1.0       0.84      0.86      0.85       548

avg / total       0.85      0.85      0.85      1082



In [37]:
# evaluate the model using 10-fold cross-validation
scores = cross_val_score(LDA(), X, y, scoring='accuracy', cv=10)
print (scores)
print (scores.mean())

[ 0.94095941  0.79335793  0.75645756  0.85555556  0.85925926  0.83333333
  0.82222222  0.82592593  0.87037037  0.82962963]
0.838707120405


/home/motech/anaconda/lib/python2.7/site-packages/sklearn/lda.py:371: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


###KNN

In [10]:
# KNN
model = KNeighborsClassifier()
model = model.fit(X, y)

# check the score
model.score(X, y)

0.85275619681835002

In [11]:
# evaluate the model by splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=1)
model2 = KNeighborsClassifier()
model2.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=5, p=2, weights='uniform')

In [12]:
# predict class labels for the test set
predicted = model2.predict(X_test)
print (predicted)

[ 1.  1.  1. ...,  1.  1.  0.]


In [13]:
# generate class probabilities
probs = model2.predict_proba(X_test)
print (probs)

[[ 0.   1. ]
 [ 0.   1. ]
 [ 0.   1. ]
 ..., 
 [ 0.2  0.8]
 [ 0.4  0.6]
 [ 1.   0. ]]


In [14]:
# generate evaluation metrics
print (metrics.accuracy_score(y_test, predicted))
print (metrics.roc_auc_score(y_test, probs[:, 1]))

0.792975970425
0.852613521419


In [15]:
# print confusion matrix
print (metrics.confusion_matrix(y_test, predicted))
print (metrics.classification_report(y_test, predicted))

[[418 116]
 [108 440]]
             precision    recall  f1-score   support

        0.0       0.79      0.78      0.79       534
        1.0       0.79      0.80      0.80       548

avg / total       0.79      0.79      0.79      1082



In [16]:
# evaluate the model using 10-fold cross-validation
scores = cross_val_score(KNeighborsClassifier(), X, y, scoring='accuracy', cv=10)
print (scores)
print (scores.mean())

[ 0.80442804  0.75645756  0.73431734  0.83333333  0.84444444  0.78888889
  0.8037037   0.82592593  0.81851852  0.76666667]
0.797668443351


###Automate analysis to run on multiple sets of features

In [3]:
# set up 3 datasets for automation
# and remove NaNs

# dataset 1
df_fix1 = df[~np.isnan(df.Adult_smoking_Value) & ~np.isnan(df.Adult_obesity_Value) &
           ~np.isnan(df.Physical_inactivity_Value) & ~np.isnan(df.Diabetes_Value) &
            ~np.isnan(df.Teen_births_Value) & ~np.isnan(df.aboveMedianPDV)]
df_fix1 = df_fix1.reset_index(drop=True)

# create dataframes with an intercept column
y1, X1 = dmatrices('aboveMedianPDV ~ Adult_smoking_Value + Adult_obesity_Value + \
                 Physical_inactivity_Value + Diabetes_Value + Teen_births_Value',
                df_fix1, return_type="dataframe")
# flatten y into a 1-D array
y1 = y1.values.flatten()

# dataset 2
df_fix2 = df[~np.isnan(df.Children_in_poverty_Value) & ~np.isnan(df.Income_inequality_Value) &
           ~np.isnan(df.Unemployment_Value) & ~np.isnan(df.Severe_housing_problems_Value) &
            ~np.isnan(df.Social_associations_Value) & ~np.isnan(df.aboveMedianPDV)]
df_fix2 = df_fix2.reset_index(drop=True)

# create dataframes with an intercept column
y2, X2 = dmatrices('aboveMedianPDV ~ Children_in_poverty_Value + Income_inequality_Value + \
                 Unemployment_Value + Severe_housing_problems_Value + Social_associations_Value',
                df_fix2, return_type="dataframe")
# flatten y into a 1-D array
y2 = y2.values.flatten()

# dataset 1
df_fix3 = df[~np.isnan(df.Food_insecurity_Value) & ~np.isnan(df.Limited_access_to_healthy_foods_Value) &
           ~np.isnan(df.Median_household_income_Value) & ~np.isnan(df.Health_care_costs_Value) &
            ~np.isnan(df.Population_living_in_a_rural_area_Value) & ~np.isnan(df.aboveMedianPDV)]
df_fix3 = df_fix3.reset_index(drop=True)

# create dataframes with an intercept column
y3, X3 = dmatrices('aboveMedianPDV ~ Food_insecurity_Value + Limited_access_to_healthy_foods_Value + \
                 Median_household_income_Value + Health_care_costs_Value + Population_living_in_a_rural_area_Value',
                df_fix3, return_type="dataframe")
# flatten y into a 1-D array
y3 = y3.values.flatten()

datasets = {1:[X1,y1], 2:[X2,y2], 3:[X3,y3]}

In [17]:
# prepare functions for automation
import sklearn
logistic = sklearn.linear_model.LogisticRegression()
lda = sklearn.lda.LDA()
knn = sklearn.neighbors.KNeighborsClassifier()
functions = {1:logistic, 2:lda, 3:knn}
ks = [5, 15, 25]

In [36]:
# function to fit models
def fitModel(datadict, functiondict):
    for key in datadict:
        for fun_key in functiondict:
            pair = datadict[key]
            X = pair[0]
            y = pair[1]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=1)
            model = functiondict[fun_key]
            model = model.fit(X_train, y_train)
            predicted = model.predict(X_test)
            # generate class probabilities
            probs = model.predict_proba(X_test)
            # model accuracy
            #print (metrics.accuracy_score(y_test, predicted))
            # confusion matrix
            print ('\n')
            print(functiondict[fun_key].__class__)
            print('\n'+" --> dataset %d " %(key))
            print ('\n'+ "Confusion matrix:"+'\n')
            print(metrics.confusion_matrix(y_test, predicted))
            # evaluate the model using 10-fold cross-validation
            scores = cross_val_score(functiondict[fun_key], X, y, scoring='accuracy', cv=10)
            print ('\n'+"The model score:"+'\n')
            print(scores.mean())
    return

In [35]:
fitModel(datasets, functions)



<class 'sklearn.linear_model.logistic.LogisticRegression'>

 --> dataset 1 

Confusion matrix:

[[432 102]
 [ 88 460]]

The model score:

0.82651769851


<class 'sklearn.lda.LDA'>

 --> dataset 1 

Confusion matrix:

[[442  92]
 [ 74 474]]

The model score:

0.838707120405


<class 'sklearn.neighbors.classification.KNeighborsClassifier'>

 --> dataset 1 

Confusion matrix:

[[418 116]
 [108 440]]

The model score:

0.797668443351


<class 'sklearn.linear_model.logistic.LogisticRegression'>

 --> dataset 2 

Confusion matrix:

[[463 156]
 [ 90 511]]

The model score:

0.789032937736


<class 'sklearn.lda.LDA'>

 --> dataset 2 

Confusion matrix:

[[464 155]
 [ 81 520]]

The model score:

0.797232972136


<class 'sklearn.neighbors.classification.KNeighborsClassifier'>

 --> dataset 2 

Confusion matrix:

[[400 219]
 [150 451]]

The model score:

0.678459322325


<class 'sklearn.linear_model.logistic.LogisticRegression'>

 --> dataset 3 

Confusion matrix:

[[469 142]
 [118 489]]

The m

###Project Idea investigation : 
####Round 1.  Locate a data source that interests you (consider Datasets and Project Ideas link).  What data fields would be involved in your analysis ? Describe some of the Machine Learning techniques that could be used to analyzed this data. My expectation is that it will require at least a couple of paragraphs to address the above. 

###Project Partner --> Sreedevi Gangarapu

###Project Dataset: Yelp Dataset (from Yelp Dataset Challenge)

    For our final project, Sreedevi and I decided to use the Yelp dataset with a possibility of submitting our work as a solution to their challenge. They offer prizes to students who can use their dataset in a creative and meaningful way.
    The organizers of the challenge provide participants with some initial ideas and starting points that help decide what questions we want answered with the help of this dataset. We currently feel very interested in their suggestions for Natural Language Processing. There are several questions that Yelp organizers are interested in. First, we could try to predict the rating of the review from its text, which would require using some sentiment analysis technique or tool. Another question that interests Yelp is whether their reviewers are sarcastic. We found a description of a method someone used to determine whether an expression is colored with sarcasm. It requires collecting sentences from Twitter with a #sarcasm and using them for training of the "sarcasm detector". It is an interesting path for a project, but can be too complex for this class. Finally, The challenge organizers are interested in the most common positive an negative words used by their reviewers. That could be don using several sentiment analysis tools.
    The dataset consists of 1.6M reviews and 500K tips by 366K users for 61K businesses with 481K business attributes, e.g., hours, parking availability, ambience, social network of 366K users for a total of 2.9M social edges, and aggregated check-ins over time for each of the 61K businesses. There are a lot of othere interesting questions that can be answered with dataset about seasonal and cultural trends. We are considering some of these question also potentially interesting and useful for this project.
